# 📘 DevOps 업무전환 - Story 생성
- **목적**: DevOps 전환을 위해 생성된 에픽에서 스토리 생성, 생성된 스토리의 우선순위 선정
- **입력 데이터**: generate_epic_of_devops.ipynb의 결과물
- **주요 기능**
  - 에픽에서 스토리 생성
  - 스토리 우선 순위 산정
- **작성자**: 김동영
- **작성일**: 2025-08-01

# 0. 환경 설정

In [1]:
import os, sys
sys.path.append(os.path.abspath("../src"))

import dotenv
import json

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

from story.routes import StoryGeneratorAgent
# from epic.prompts import TASK_TO_EPIC_CONVERTER_PROMPT

/Users/junkiwon/Desktop/IOL/work directory/project_manage_agent/src/story/services.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(


In [2]:
# .env 환경번수 load
dotenv.load_dotenv('../.env')

True

# 1. Agent 생성 및 스토리 생성

In [3]:
# 스토리 생성 llm 모델 생성
story_agent = StoryGeneratorAgent(
    openai_api_key = os.getenv("OPENAI_API_KEY")
)

In [5]:
STORY_GENERATOR_PROMPT = ChatPromptTemplate.from_messages(
    [SystemMessagePromptTemplate.from_template(
        """
        [Role] 너는 숙련된 스크럼 마스터다.
        [Goal] 2주 단위 스프린트를 기반으로 한 애자일 방법론에서, 프로젝트 정보와 Epic에 따라 실제로 필요한 Story를 도출하고 업무 계획을 수립하는 데 도움을 주는 것.
        [Instruction] 주어진 프로젝트 정보와 Epic을 기반으로, Epic를 구성하는 하나 이상의 Story을 생성해.
        [Constraints]
          - 출력은 반드시 [Output Format]을 따라야 한다.
          - Story는 하나의 스프린트 기간 내에 완료될 수 있는 작업 단위여야 한다.
          - 불필요한 설명, 주석, 마크다운 없이 JSON만 출력한다.
          - 최대 {max_storys}개의 Story을 생성한다.
          - 해당 에픽을 완수하기 위한 최소한의 Story만을 생성한다.
          - acceptance_criteria는 구체적으고 측정 가능한 기준으로 작성한다.
          - 에픽에 테스트에 대한 내용이 없으면 테스트 항목은 제외한다.
          - domain은 해당 스토리의 주요 개발 영역을 명시한다.
          - story_type은 스토리의 성격을 분류한다.
          - tags는 관련 기술스택, 기능영역, 키워드를 포함한다.
        [Process]
        1. Epic 분석: 주어진 Epic의 범위와 목표를 파악한다
        2. Story 도출: Epic을 완료하기 위해 필요한 독립적인 작업 단위들을 식별한다
        3. 우선순위 결정: 비즈니스 가치와 의존성을 고려하여 우선순위를 설정한다
        4. 수락 기준 정의: 각 Story의 완료 조건을 구체적으로 명시한다
        [Output Format]
        [
          {{
            "title": "스토리 제목",
            "description": "스토리 설명", 
            "acceptance_criteria": ["수락 기준 1", "수락 기준 2", "수락 기준 3"],
            "domain":  "frontend|backend|devops|data",
            "story_type": "feature|bug|refactor|enhancement|documentation",
            "tags": ["관련 키워드", "기술스택", "기능영역"]
          }},
        ]
        """
    ),
    HumanMessagePromptTemplate.from_template(
        """
        [Input] {user_input}
        [Context] {epic_info}
        """
    )
    ]
)


In [4]:
# epic_info
epic_info = {
    "title": "CI/CD 자동화 구성",
    "description": "GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.",
    "business_value": "개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.",
    "priority": "High",
    "acceptance_criteria": [
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.",
      "Dockerfile 실행 오류 및 종속성 검증이 포함된다.",
      "CI/CD 결과가 Slack/이메일/웹훅으로 알림된다."
    ],
    "included_tasks": [
      "GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성",
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행",
      "Dockerfile 실행 오류 및 종속성 검증 포함",
      "CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동"
    ]
  },

In [ ]:
generated_storys_str = story_agent._generate_storys_with_llm(
    prompt = STORY_GENERATOR_PROMPT,
    user_input = '',
    epic_info= epic_info,
    max_storys=10
)

In [ ]:
parsed = json.loads(generated_storys_str)
parsed

In [19]:
epic_aws_resource_info =       {
      "title":"보안 정책 및 기준 설정",
      "description":"인프라 리소스와 관련된 보안 정책 및 기준을 정의하는 작업",
      "business_value":"보안 리스크 감소 및 규정 준수",
      "priority":"Medium",
      "acceptance_criteria":[
         "모든 보안 정책이 문서화되어 있으며 주기적으로 검토됨",
         "민감정보 관리 기준이 정의되어 있음"
      ],
      "included_tasks":[
         "정책 생성 (인프라 리소스 네이밍 룰 정의, 키 관리방법, MFA, 보안 그룹 등 보안 정책 명시, 주기적 비용 리포트 및 미사용 자원 정리 기준, 민감정보 관리 기준 정책 등)"
      ]
   }


generated_storys_aws_resource_str= story_agent._generate_storys_with_llm(
    prompt = STORY_GENERATOR_PROMPT,
    user_input = '',
    epic_info= epic_aws_resource_info,
    max_storys=10
)
parsed = json.loads(generated_storys_aws_resource_str)
parsed

INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료


[{'title': '인프라 리소스 네이밍 룰 정의',
  'description': '인프라 리소스의 네이밍 규칙을 정의하고 문서화한다.',
  'acceptance_criteria': ['네이밍 룰이 문서화되어 팀원에게 공유됨',
   '정의된 네이밍 룰이 모든 인프라 리소스에 적용됨',
   '주기적으로 네이밍 룰이 검토되고 업데이트됨'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['보안 정책', '인프라', '네이밍 규칙']},
 {'title': '키 관리 방법 정의',
  'description': '인프라 리소스에서 사용할 키 관리 방법을 정의하고 문서화한다.',
  'acceptance_criteria': ['키 관리 방법이 문서화되어 팀원에게 공유됨',
   '정의된 키 관리 방법이 모든 관련 시스템에 적용됨',
   '주기적으로 키 관리 방법이 검토되고 업데이트됨'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['보안 정책', '키 관리', '인프라']},
 {'title': 'MFA 정책 정의',
  'description': '다단계 인증(MFA) 정책을 정의하고 문서화한다.',
  'acceptance_criteria': ['MFA 정책이 문서화되어 팀원에게 공유됨',
   '정의된 MFA 정책이 모든 사용자 계정에 적용됨',
   '주기적으로 MFA 정책이 검토되고 업데이트됨'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['보안 정책', 'MFA', '인프라']},
 {'title': '보안 그룹 정책 정의',
  'description': '인프라 리소스의 보안 그룹 정책을 정의하고 문서화한다.',
  'acceptance_criteria': ['보안 그룹 정책이 문서화되어 팀원에게 공유됨',
   '정의된 보안 그룹 정책이 모든 관련

# 에픽별 스토리 산출

In [11]:
epics_list = [{
    "title": "CI/CD 자동화 구성",
    "description": "GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.",
    "business_value": "개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.",
    "priority": "High",
    "acceptance_criteria": [
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.",
      "Dockerfile 실행 오류 및 종속성 검증이 포함된다.",
      "CI/CD 결과가 Slack/이메일/웹훅으로 알림된다."
    ],
    "included_tasks": [
      "GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성",
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행",
      "Dockerfile 실행 오류 및 종속성 검증 포함",
      "CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동"
    ]
  },{
	  "title": "브랜치 보호 및 환경/비밀 관리",
	  "description": "코드 품질을 보장하기 위한 브랜치 보호 설정과, 각 환경별 구성 및 민감 정보의 안전한 관리를 수행한다.",
	  "business_value": "협업 효율성과 코드 안정성을 높이고, 환경별 배포 신뢰성을 강화한다.",
	  "priority": "Medium",
	  "acceptance_criteria": [
	    "PR 병합은 main, develop 브랜치로 제한된다.",
	    "브랜치 보호 규칙이 적용되어 있다.",
	    "각 브랜치/환경에 대해 별도 설정 파일 및 비밀 저장소가 관리된다."
	  ],
	  "included_tasks": [
	    "main, develop 브랜치에 대한 병합 제한 규칙 설정",
	    "브랜치 보호 규칙(리뷰 승인, 상태 확인 등) 적용",
	    ".env 파일, Secrets Manager, Parameter Store를 통한 환경 및 비밀 설정 구성",
	    "환경별 배포 구성 및 변수 관리 문서화"
	  ]
 },{
    "title": "Docker 이미지 관리",
    "description": "CI/CD 프로세스에서 Docker 이미지를 자동으로 빌드하고 푸시하는 기능을 구현하고, 이미지 생성 및 관리 방안을 정립한다.",
    "business_value": "애플리케이션 배포의 일관성을 유지하고, 배포 속도를 향상시킨다.",
    "priority": "High",
    "acceptance_criteria": [
      "CodeBuild와 연동하여 buildspec.yml이 사용된다.",
      "DockerHub 또는 ECR에 결과 이미지가 자동으로 푸시된다."
    ],
    "included_tasks": [
      "어플리케이션 이미지 단위 패키징 정책 논의",
      "DockerHub 또는 ECR에 결과 이미지 자동 푸시",
      "Docker Image Build 자동화",
      "Docker image build 시 테스트 방안 논의"
    ]
  },
  {
  "title": "배포 전략 및 운영 안정성 관리",
  "description": "다양한 배포 전략, 보안 설정, 헬스체크 및 롤백을 통합하여 운영 중단 없이 안전하고 신뢰성 있는 배포를 지원한다.",
  "business_value": "서비스 중단 없는 배포를 가능하게 하여 사용자 경험을 보호하고, 운영 리스크를 최소화한다.",
  "priority": "High",
  "acceptance_criteria": [
    "배포 트리거가 GitHub Webhook, PR 병합, 수동 승인 등으로 설정된다.",
    "배포 전략이 블루그린, 카나리아, 롤링 중 하나 이상으로 설정된다.",
    "AWS IAM Role 및 SSH 키를 통한 보안 배포가 구성된다.",
    "배포 전후 헬스체크가 수행되고, 실패 시 자동 롤백이 작동한다."
  ],
  "included_tasks": [
    "배포 트리거 전략 수립 및 적용(GitHub Webhook, 수동 승인 등)",
    "배포 전략 적용(블루그린, 카나리아, 롤링 중 선택)",
    "AWS IAM Role 및 SSH 키를 통한 안전한 배포 구성",
    "배포 전/후 헬스체크 구현 및 실패 시 자동 롤백 설정",
    "배포 전략 및 롤백 구성 관련 문서화"
  ]
},
  {
  "title": "운영 정책 및 배포 이력 관리",
  "description": "배포 정책과 템플릿을 표준화하고, 배포 이력 및 승인 기록을 체계적으로 관리하여 프로젝트의 일관성과 추적 가능성을 확보한다.",
  "business_value": "운영 프로세스를 표준화하여 품질을 유지하고, 문제 발생 시 신속한 원인 파악 및 대응이 가능하도록 한다.",
  "priority": "Medium",
  "acceptance_criteria": [
    "배포 이력 및 승인 내역이 저장되고 관리된다.",
    "워크플로 및 파이프라인의 명명 규칙이 명확히 정의되어 있다.",
    "신규 프로젝트에 사용할 수 있는 배포 정책 템플릿과 체크리스트가 제공된다."
  ],
  "included_tasks": [
    "배포 이력 및 승인 기록 저장 시스템 구성",
    "운영 및 배포 프로세스에 대한 정책 문서화",
    "워크플로 및 파이프라인 명명 규칙 정의",
    "신규 프로젝트용 배포 템플릿 및 체크리스트 작성 및 배포"
  ]
}]

In [12]:
storys_by_epic = {}

for epic in epics_list[1:]:
    storys_by_epic[epic['title']] = {}

    generated_storys_str = story_agent._generate_storys_with_llm(
        prompt = STORY_GENERATOR_PROMPT,
        user_input = '',
        epic_info= epic,
        max_storys=10
    )
    parsed = json.loads(generated_storys_str)
    storys_by_epic[epic['title']] = parsed


INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료


In [15]:
storys_by_epic['운영 정책 및 배포 이력 관리']

[{'title': '배포 이력 및 승인 기록 저장 시스템 구성',
  'description': '배포 이력과 승인 기록을 저장하고 관리할 수 있는 시스템을 구성한다.',
  'acceptance_criteria': ['배포 이력과 승인 기록이 데이터베이스에 저장된다.',
   '저장된 이력은 조회 및 필터링이 가능하다.',
   '시스템이 배포 이력의 자동 기록 기능을 제공한다.'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['배포 관리', '데이터베이스', '운영 프로세스']},
 {'title': '운영 및 배포 프로세스에 대한 정책 문서화',
  'description': '운영 및 배포 프로세스에 대한 정책을 문서화하여 팀원들이 쉽게 접근할 수 있도록 한다.',
  'acceptance_criteria': ['정책 문서가 팀의 위키 또는 문서 저장소에 업로드된다.',
   '문서에는 배포 절차와 승인 프로세스가 포함된다.',
   '문서가 팀원들에게 공유되고 피드백을 받는다.'],
  'domain': 'documentation',
  'story_type': 'documentation',
  'tags': ['문서화', '운영 정책', '배포 프로세스']},
 {'title': '워크플로 및 파이프라인 명명 규칙 정의',
  'description': '배포 워크플로 및 파이프라인의 명명 규칙을 정의하여 일관성을 유지한다.',
  'acceptance_criteria': ['명명 규칙이 문서화되어 팀원들에게 배포된다.',
   '명명 규칙이 실제 배포 파이프라인에 적용된다.',
   '팀원들이 명명 규칙을 준수하고 있는지 검토한다.'],
  'domain': 'devops',
  'story_type': 'feature',
  'tags': ['명명 규칙', 'CI/CD', '운영 프로세스']},
 {'title': '신규 프로젝트용 배포 템플릿 및 체크리스트 작성',
  'des

In [7]:
infra_epics_list = [
   {
      "title":"AWS 리소스 관리 및 비용 최적화",
      "description":"기존 사용 중인 AWS 리소스를 관리하고 비용 최적화 방안을 모색하는 작업",
      "business_value":"비용 절감 및 리소스 효율성 향상",
      "priority":"High",
      "acceptance_criteria":[
         "모든 미사용 AWS 리소스가 식별되고 정리되었음",
         "비용 최적화 방안이 문서화되어 있음",
         "리소스 태깅 정책이 적용되어 예산 분류가 가능함"
      ],
      "included_tasks":[
         "EC2, ECS, EKS, S3, RDS, ALB 등 기존 사용 중인 AWS 리소스를 관리하고 주기적으로 미사용 리소스를 정리하고 분석하여 비용 최적화 방안을 모색",
         "환경별 구성 분리, 리소스 태깅",
         "Auto Scaling, 예약 인스턴스, spot 인스턴스, 인스턴스 사양 관리 등 비용 최적화 방안을 찾음",
         "리소스 태깅 정책 적용 및 예산 분류 가능하게 구성"
      ]
   },
   {
      "title":"인증서 관리 및 자동 갱신",
      "description":"SSL 인증서 및 도메인 관리와 관련된 업무",
      "business_value":"서비스의 보안성 강화 및 운영 효율성 증대",
      "priority":"Medium",
      "acceptance_criteria":[
         "모든 SSL 인증서가 자동으로 갱신되도록 설정됨",
         "도메인 관리 프로세스가 문서화되어 있음"
      ],
      "included_tasks":[
         "인증서 자동 갱신 (ACM, certbot 등)",
         "Route53, 도메인, SSL 인증서 갱신 관리"
      ]
   },
   {
      "title":"IAM 및 접근 제어 관리",
      "description":"IAM Role 기반의 최소 권한 접근 제어 및 정책 관리",
      "business_value":"보안 강화 및 접근 관리 효율성 증대",
      "priority":"High",
      "acceptance_criteria":[
         "모든 IAM role, 정책, 사용자가 정리되고 관리됨",
         "접근 제어 정책이 문서화되어 있음"
      ],
      "included_tasks":[
         "IAM Role 기반 최소 권한 접근 제어를 하도록 구성하고 기존 IAM role, 정책, 사용자를 정리하고 관리",
         "파트너사와의 역할 분리 및 접근 제어 정책 관리 (접속 권한, 키 회수 등)"
      ]
   },
   {
      "title":"보안 정책 및 기준 설정",
      "description":"인프라 리소스와 관련된 보안 정책 및 기준을 정의하는 작업",
      "business_value":"보안 리스크 감소 및 규정 준수",
      "priority":"Medium",
      "acceptance_criteria":[
         "모든 보안 정책이 문서화되어 있으며 주기적으로 검토됨",
         "민감정보 관리 기준이 정의되어 있음"
      ],
      "included_tasks":[
         "정책 생성 (인프라 리소스 네이밍 룰 정의, 키 회수, MFA, 보안 그룹 등 보안 정책 명시, 주기적 비용 리포트 및 미사용 자원 정리 기준, 민감정보 관리 기준 정책 등)"
      ]
   }
]

In [13]:
storys_by_epic = {}

for epic in infra_epics_list:
    storys_by_epic[epic['title']] = {}

    generated_storys_str = story_agent._generate_storys_with_llm(
        prompt = STORY_GENERATOR_PROMPT,
        user_input = '',
        epic_info= epic,
        max_storys=10
    )
    parsed = json.loads(generated_storys_str)
    storys_by_epic[epic['title']] = parsed

INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료


In [9]:
monitoring_and_log_epic_list = [
   {
      "title":"메트릭 및 로그 대시보드 구성",
      "description":"Prometheus와 Grafana를 활용하여 메트릭 대시보드를 구성하고, Loki와 Grafana를 통해 서비스 로그를 수집 및 시각화하는 작업.",
      "business_value":"서비스의 성능 및 상태를 실시간으로 모니터링하여 운영 효율성을 향상.",
      "priority":"High",
      "acceptance_criteria":[
         "Prometheus와 Grafana를 통해 메트릭 대시보드가 성공적으로 구성됨.",
         "Loki와 Grafana를 통해 서비스 로그가 수집되고 시각화됨.",
         "대시보드에서 주요 메트릭 및 로그가 실시간으로 표시됨."
      ],
      "included_tasks":[
         "Prometheus + Grafana로 메트릭 대시보드 구성",
         "Loki + Grafana로 서비스 중인 서버의 로그 수집 및 시각화"
      ]
   },
   {
      "title":"로그 관리 및 정책 수립",
      "description":"서비스 로그의 수집, 저장, 보존 정책을 수립하고, 로그 스키마 및 레벨을 통일하는 작업.",
      "business_value":"일관된 로그 관리로 데이터 분석 및 문제 해결의 효율성을 높임.",
      "priority":"High",
      "acceptance_criteria":[
         "로그 스키마와 레벨이 JSON 기반으로 통일됨.",
         "로그 보존 주기 및 저장 정책이 정의됨.",
         "서비스별 로그 레벨 및 포맷 표준화 문서가 작성됨."
      ],
      "included_tasks":[
         "로그 스키마/레벨 통일, JSON 기반 구조로 통일",
         "로그 보존 주기 및 저장 정책 수립",
         "정책 생성(서비스별 로그 레벨 및 포맷 표준화, 알람 임계치 설정 기준 문서화, 로그 보관 주기 및 백업 연계 정의 등)"
      ]
   },
   {
      "title":"알림 및 통합 시스템 구축",
      "description":"Slack과의 알림 연동 및 다양한 로그 관리 도구와의 연동 방안을 모색하는 작업.",
      "business_value":"신속한 문제 대응 및 팀 간의 원활한 커뮤니케이션을 지원.",
      "priority":"Medium",
      "acceptance_criteria":[
         "Slack으로의 알림 연동이 성공적으로 구현됨.",
         "Datadog, ELK, CloudWatch Logs 등과의 연동 방안이 문서화됨.",
         "주요 이벤트(에러, 로그인, 배포 등)가 자동으로 수집되고 알림이 전송됨."
      ],
      "included_tasks":[
         "Slack으로 알림연동",
         "Datadog, ELK, CloudWatch Logs 등과 연동하는 방안 모색",
         "주요 이벤트(에러, 로그인, 배포 등) 자동 수집 및 보관 방법 모색 및 적용"
      ]
   },
   {
      "title":"보안 및 개인정보 보호 정책",
      "description":"개인정보 및 보안 정보를 마스킹 처리하는 방안을 통일하고 적용하는 작업.",
      "business_value":"사용자 개인정보 보호 및 보안 강화를 통해 법적 요구사항을 준수.",
      "priority":"High",
      "acceptance_criteria":[
         "개인정보 및 보안 정보의 마스킹 처리 방안이 통일됨.",
         "마스킹 처리 기준이 문서화되고 적용됨.",
         "모든 로그에서 개인정보가 안전하게 처리됨."
      ],
      "included_tasks":[
         "개인정보/보안 정보 마스킹 처리 방안 통일"
      ]
   }
]

In [10]:
storys_by_epic = {}

for epic in monitoring_and_log_epic_list:
    storys_by_epic[epic['title']] = {}

    generated_storys_str = story_agent._generate_storys_with_llm(
        prompt = STORY_GENERATOR_PROMPT,
        user_input = '',
        epic_info= epic,
        max_storys=10
    )
    parsed = json.loads(generated_storys_str)
    storys_by_epic[epic['title']] = parsed

INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료


In [13]:
container_epic_list = [
   {
      "title":"Docker 환경 설정 및 관리",
      "description":"Docker 환경을 설정하고 관리하기 위한 작업들을 포함합니다.",
      "business_value":"효율적인 개발 환경 구축 및 운영",
      "priority":"High",
      "acceptance_criteria":[
         "Dockerfile 멀티스테이지 빌드가 성공적으로 구성되어야 한다.",
         "Docker Compose를 통해 환경별 서비스가 분리되어야 한다.",
         "환경변수 관리 방식이 설정되어야 한다."
      ],
      "included_tasks":[
         "Dockerfile 멀티스테이지 빌드 구성",
         "Docker Compose로 환경별 서비스 분리",
         "환경변수 관리 방식 설정( `.env`, Secrets Manager, ConfigMap 등)"
      ]
   },
   {
      "title":"컨테이너 네트워크 및 데이터 관리",
      "description":"컨테이너의 네트워크 및 데이터 영속성을 관리하는 작업들을 포함합니다.",
      "business_value":"안정적인 데이터 관리 및 네트워크 구성",
      "priority":"Medium",
      "acceptance_criteria":[
         "컨테이너 네트워크가 브릿지, 포트 바인딩, DNS 구성이 완료되어야 한다.",
         "볼륨 마운트 및 데이터 영속화 정책이 수립되어야 한다."
      ],
      "included_tasks":[
         "컨테이너 네트워크 브릿지, 포트 바인딩, DNS 구성",
         "볼륨 마운트 및 데이터 영속화 정책"
      ]
   },
   {
      "title":"Kubernetes 클러스터 구성 및 관리 또는 Kubernetes 미적용",
      "description":"Kubernetes 클러스터를 구성하고 관리하기 위한 작업들을 포함합니다.",
      "business_value":"확장성과 관리 용이성을 제공하는 클라우드 네이티브 환경 구축",
      "priority":"High",
      "acceptance_criteria":[
         "Kubernetes 적용 여부가 판단되어야 하며, 적용 시 클러스터가 성공적으로 구성되어야 한다.",
         "Deployment, Service, Ingress 등의 리소스 정의가 완료되어야 한다.",
         "Helm Chart로 템플릿화가 완료되어야 하며, HPA가 적용되어야 한다."
      ],
      "included_tasks":[
         "Kubernetes 적용 여부 판단 후 적용 또는 미적용(`eksctl`, `Helm` 사용한 클러스터 구성, Deployment, Service, Ingress 등 리소스 정의, Helm Chart로 템플릿화, HPA (Horizontal Pod Autoscaler) 적용 등)"
      ]
   },
   {
      "title":"배포 정책 및 보안 관리",
      "description":"배포 리소스와 관련된 정책 및 보안 관리 작업들을 포함합니다.",
      "business_value":"안전하고 일관된 배포 환경 구축",
      "priority":"Medium",
      "acceptance_criteria":[
         "배포 리소스 네이밍 가이드가 작성되어야 한다.",
         "Secret, ConfigMap의 분리 저장 및 암호화 기준이 수립되어야 한다.",
         "이미지 취약점 검사 및 업데이트 주기가 문서화되어야 한다."
      ],
      "included_tasks":[
         "정책 생성(배포 리소스 네이밍 가이드 (Helm Chart, k8s 리소스 등), Secret, ConfigMap 분리 저장 및 암호화 기준, 이미지 취약점 검사 및 업데이트 주기 문서화 등)"
      ]
   }
]

In [ ]:
storys_by_epic = {}

for epic in container_epic_list:
    storys_by_epic[epic['title']] = {}

    generated_storys_str = story_agent._generate_storys_with_llm(
        prompt = STORY_GENERATOR_PROMPT,
        user_input = '',
        epic_info= epic,
        max_storys=10
    )
    parsed = json.loads(generated_storys_str)
    storys_by_epic[epic['title']] = parsed

INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료


In [18]:
iac_epic_list = [
   {
      "title":"인프라 관리 자동화",
      "description":"AWS 리소스를 Terraform 또는 AWS CDK로 관리하고, CI/CD 연동을 통해 인프라 변경을 자동화하는 Epic입니다.",
      "business_value":"인프라 관리의 효율성을 높이고, 변경 사항의 안정성을 확보합니다.",
      "priority":"High",
      "acceptance_criteria":[
         "기존 AWS 리소스를 Terraform 또는 AWS CDK로 코드화 완료",
         "CI/CD 연동을 통해 IaC 자동 적용 프로세스 구축",
         "PR 생성 시 terraform plan 결과 리뷰 및 테스트 환경 자동 배포 기능 구현"
      ],
      "included_tasks":[
         "기존 AWS 리소스를 포함하여 인프라를 선언형 코드로 관리하도록 전환",
         "CI/CD 연동으로 IaC 자동 적용",
         "PR 생성 시 terraform plan 결과 리뷰 → 테스트 환경 자동 배포(Sandbox) → smoke test → 승인 후 apply 진행"
      ]
   },
   {
      "title":"모듈화 및 환경 관리",
      "description":"인프라 코드의 모듈화 및 환경별 Workspace 분리를 통해 관리의 일관성을 높이는 Epic입니다.",
      "business_value":"환경별 관리의 용이성을 제공하고, 코드의 재사용성을 높입니다.",
      "priority":"Medium",
      "acceptance_criteria":[
         "모듈화 및 환경별 Workspace 분리 적용 완료",
         "디렉토리 구조/변수/출력 표준화 가이드 문서화 완료"
      ],
      "included_tasks":[
         "모듈화 및 환경별 Workspace 분리 적용 (dev, staging, prod) 및 디렉토리 구조/변수/출력 표준화 가이드 문서화"
      ]
   },
   {
      "title":"보안 정책 및 관리",
      "description":"인프라의 보안을 강화하기 위한 정책을 정의하고 자동화하는 Epic입니다.",
      "business_value":"보안 리스크를 줄이고, 인프라의 안전성을 높입니다.",
      "priority":"High",
      "acceptance_criteria":[
         "IAM Role 및 Policy 정의를 코드로 관리 완료",
         "민감 정보는 AWS Secrets Manager / SSM Parameter Store로 분리 및 Secret 자동 Rotate 정책 적용"
      ],
      "included_tasks":[
         "보안 정책 내재화 (IAM Role 및 Policy 정의를 코드로 관리, Security Group, NACL, S3 정책 등 보안 설정 자동화)"
      ]
   },
   {
      "title":"정책 및 규칙 관리",
      "description":"인프라 리소스의 명명 규칙 및 접근 권한을 정의하고 관리하는 Epic입니다.",
      "business_value":"조직 내 인프라 관리의 일관성을 높입니다.",
      "priority":"Low",
      "acceptance_criteria":[
         "IaC 리소스 명명 규칙 및 접근 권한 분리 정책 정의 완료",
         "정기적인 모듈 업데이트 및 리팩토링 주기 정의 완료"
      ],
      "included_tasks":[
         "정책 생성 (IaC 리소스 명명 규칙, 접근 권한 분리 등)"
      ]
   }
]

In [19]:
storys_by_epic = {}

for epic in iac_epic_list:
    storys_by_epic[epic['title']] = {}

    generated_storys_str = story_agent._generate_storys_with_llm(
        prompt = STORY_GENERATOR_PROMPT,
        user_input = '',
        epic_info= epic,
        max_storys=10
    )
    parsed = json.loads(generated_storys_str)
    storys_by_epic[epic['title']] = parsed

INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료


In [21]:
ci_cd_epic_list = [
  {
    "title": "CI/CD 자동화 구성",
    "description": "GitHub Actions 또는 Jenkins를 활용하여 End-to-End CI/CD 자동화 프로세스를 구성한다.",
    "business_value": "개발 및 배포 프로세스의 효율성을 높이고, 코드 품질을 유지하여 빠른 피드백을 제공한다.",
    "priority": "High",
    "acceptance_criteria": [
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check가 자동으로 실행된다.",
      "Dockerfile 실행 오류 및 종속성 검증이 포함된다.",
      "CI/CD 결과가 Slack/이메일/웹훅으로 알림된다."
    ],
    "included_tasks": [
      "GitHub Actions 또는 Jenkins를 활용한 End-to-End 자동화 구성",
      "PR 생성 시 Lint, Unit Test, Build, Code Quality Check 자동 실행",
      "Dockerfile 실행 오류 및 종속성 검증 포함",
      "CI/CD 결과를 Slack/이메일/웹훅 알림으로 연동"
    ]
  },
 {
	  "title": "브랜치 보호 및 환경/비밀 관리",
	  "description": "코드 품질을 보장하기 위한 브랜치 보호 설정과, 각 환경별 구성 및 민감 정보의 안전한 관리를 수행한다.",
	  "business_value": "협업 효율성과 코드 안정성을 높이고, 환경별 배포 신뢰성을 강화한다.",
	  "priority": "Medium",
	  "acceptance_criteria": [
	    "PR 병합은 main, develop 브랜치로 제한된다.",
	    "브랜치 보호 규칙이 적용되어 있다.",
	    "각 브랜치/환경에 대해 별도 설정 파일 및 비밀 저장소가 관리된다."
	  ],
	  "included_tasks": [
	    "main, develop 브랜치에 대한 병합 제한 규칙 설정",
	    "브랜치 보호 규칙(리뷰 승인, 상태 확인 등) 적용",
	    ".env 파일, Secrets Manager, Parameter Store를 통한 환경 및 비밀 설정 구성",
	    "환경별 배포 구성 및 변수 관리 문서화"
	  ]
 },
  {
    "title": "Docker 이미지 관리",
    "description": "CI/CD 프로세스에서 Docker 이미지를 자동으로 빌드하고 푸시하는 기능을 구현한다.",
    "business_value": "애플리케이션 배포의 일관성을 유지하고, 배포 속도를 향상시킨다.",
    "priority": "High",
    "acceptance_criteria": [
      "CodeBuild와 연동하여 buildspec.yml이 사용된다.",
      "DockerHub 또는 ECR에 결과 이미지가 자동으로 푸시된다."
    ],
    "included_tasks": [
      "CodeBuild 연동 (buildspec.yml)",
      "DockerHub 또는 ECR에 결과 이미지 자동 푸시"
    ]
  },
  {
  "title": "배포 전략 및 운영 안정성 관리",
  "description": "다양한 배포 전략, 보안 설정, 헬스체크 및 롤백을 통합하여 운영 중단 없이 안전하고 신뢰성 있는 배포를 지원한다.",
  "business_value": "서비스 중단 없는 배포를 가능하게 하여 사용자 경험을 보호하고, 운영 리스크를 최소화한다.",
  "priority": "High",
  "acceptance_criteria": [
    "배포 트리거가 GitHub Webhook, PR 병합, 수동 승인 등으로 설정된다.",
    "배포 전략이 블루그린, 카나리아, 롤링 중 하나 이상으로 설정된다.",
    "AWS IAM Role 및 SSH 키를 통한 보안 배포가 구성된다.",
    "배포 전후 헬스체크가 수행되고, 실패 시 자동 롤백이 작동한다."
  ],
  "included_tasks": [
    "배포 트리거 전략 수립 및 적용(GitHub Webhook, 수동 승인 등)",
    "배포 전략 적용(블루그린, 카나리아, 롤링 중 선택)",
    "AWS IAM Role 및 SSH 키를 통한 안전한 배포 구성",
    "배포 전/후 헬스체크 구현 및 실패 시 자동 롤백 설정",
    "배포 전략 및 롤백 구성 관련 문서화"
  ]
},
  {
  "title": "운영 정책 및 배포 이력 관리",
  "description": "배포 정책과 템플릿을 표준화하고, 배포 이력 및 승인 기록을 체계적으로 관리하여 프로젝트의 일관성과 추적 가능성을 확보한다.",
  "business_value": "운영 프로세스를 표준화하여 품질을 유지하고, 문제 발생 시 신속한 원인 파악 및 대응이 가능하도록 한다.",
  "priority": "Medium",
  "acceptance_criteria": [
    "배포 이력 및 승인 내역이 저장되고 관리된다.",
    "워크플로 및 파이프라인의 명명 규칙이 명확히 정의되어 있다.",
    "신규 프로젝트에 사용할 수 있는 배포 정책 템플릿과 체크리스트가 제공된다."
  ],
  "included_tasks": [
    "배포 이력 및 승인 기록 저장 시스템 구성",
    "운영 및 배포 프로세스에 대한 정책 문서화",
    "워크플로 및 파이프라인 명명 규칙 정의",
    "신규 프로젝트용 배포 템플릿 및 체크리스트 작성 및 배포"
  ]
}
]

In [22]:
storys_by_epic = {}

for epic in ci_cd_epic_list:
    storys_by_epic[epic['title']] = {}

    generated_storys_str = story_agent._generate_storys_with_llm(
        prompt = STORY_GENERATOR_PROMPT,
        user_input = '',
        epic_info= epic,
        max_storys=10
    )
    parsed = json.loads(generated_storys_str)
    storys_by_epic[epic['title']] = parsed

INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료
INFO:story.services:스토리 생성 시작
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:story.services:스토리 생성 완료


In [24]:
storys_by_epic

{'CI/CD 자동화 구성': [{'title': 'CI/CD 자동화 도구 선택',
   'description': 'GitHub Actions 또는 Jenkins 중 하나를 선택하여 CI/CD 자동화 도구를 결정한다.',
   'acceptance_criteria': ['CI/CD 도구 선택이 완료되었다.', '선택된 도구에 대한 문서화가 이루어졌다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['CI/CD', 'GitHub Actions', 'Jenkins']},
  {'title': 'Lint 및 Unit Test 자동 실행 구성',
   'description': 'PR 생성 시 Lint 및 Unit Test가 자동으로 실행되도록 CI/CD 파이프라인을 구성한다.',
   'acceptance_criteria': ['PR 생성 시 Lint가 자동으로 실행된다.',
    'PR 생성 시 Unit Test가 자동으로 실행된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['CI/CD', 'Lint', 'Unit Test']},
  {'title': 'Build 및 Code Quality Check 자동 실행 구성',
   'description': 'PR 생성 시 Build 및 Code Quality Check가 자동으로 실행되도록 CI/CD 파이프라인을 구성한다.',
   'acceptance_criteria': ['PR 생성 시 Build가 자동으로 실행된다.',
    'PR 생성 시 Code Quality Check가 자동으로 실행된다.'],
   'domain': 'devops',
   'story_type': 'feature',
   'tags': ['CI/CD', 'Build', 'Code Quality']},
  {'title': 'Dockerfile 오류 및 종속성 검증 구성',
   'desc

In [15]:
storys_by_epic['AWS 리소스 관리 및 비용 최적화']

[{'title': '미사용 AWS 리소스 식별 및 정리',
  'description': '기존 사용 중인 AWS 리소스 중 미사용 리소스를 식별하고 정리하는 작업',
  'acceptance_criteria': ['모든 미사용 AWS 리소스가 식별되었음',
   '식별된 미사용 리소스가 정리되었음',
   '정리된 리소스 목록이 문서화되어 있음'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['AWS', '리소스 관리', '비용 최적화']},
 {'title': '비용 최적화 방안 문서화',
  'description': '비용 최적화를 위한 다양한 방안을 분석하고 문서화하는 작업',
  'acceptance_criteria': ['비용 최적화 방안이 3가지 이상 제안되었음',
   '각 방안의 장단점이 명시되었음',
   '문서화된 내용이 팀과 공유되었음'],
  'domain': 'backend',
  'story_type': 'documentation',
  'tags': ['비용 최적화', 'AWS', '문서화']},
 {'title': '리소스 태깅 정책 적용',
  'description': 'AWS 리소스에 대한 태깅 정책을 수립하고 적용하는 작업',
  'acceptance_criteria': ['리소스 태깅 정책이 정의되었음',
   '정의된 태깅 정책이 모든 리소스에 적용되었음',
   '예산 분류가 가능하도록 태깅이 완료되었음'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['AWS', '리소스 태깅', '비용 관리']},
 {'title': 'Auto Scaling 및 예약 인스턴스 설정',
  'description': '비용 최적화를 위해 Auto Scaling 및 예약 인스턴스를 설정하는 작업',
  'acceptance_criteria': ['Auto Scaling이 설정되어 트래픽 변화에 대응할 

In [18]:
storys_by_epic['보안 정책 및 기준 설정']

[{'title': '인프라 리소스 네이밍 룰 정의',
  'description': '인프라 리소스의 일관된 네이밍 룰을 정의하고 문서화한다.',
  'acceptance_criteria': ['네이밍 룰이 문서화되어 팀원에게 공유됨',
   '정의된 네이밍 룰이 모든 인프라 리소스에 적용됨',
   '주기적으로 네이밍 룰이 검토되고 업데이트됨'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['보안 정책', '인프라', '네이밍 룰']},
 {'title': '키 회수 정책 정의',
  'description': '인프라 리소스에서 사용되는 키의 회수 정책을 정의한다.',
  'acceptance_criteria': ['키 회수 정책이 문서화되어 있으며 팀원에게 공유됨',
   '정의된 정책에 따라 키 회수가 이루어짐',
   '주기적으로 키 회수 정책이 검토되고 업데이트됨'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['보안 정책', '키 관리', '정책 정의']},
 {'title': 'MFA 정책 설정',
  'description': '다단계 인증(MFA) 정책을 설정하고 문서화한다.',
  'acceptance_criteria': ['MFA 정책이 문서화되어 있으며 팀원에게 공유됨',
   '모든 사용자 계정에 MFA가 적용됨',
   '주기적으로 MFA 정책이 검토되고 업데이트됨'],
  'domain': 'backend',
  'story_type': 'feature',
  'tags': ['보안 정책', 'MFA', '인증']},
 {'title': '보안 그룹 정책 정의',
  'description': '인프라 리소스에 대한 보안 그룹 정책을 정의하고 문서화한다.',
  'acceptance_criteria': ['보안 그룹 정책이 문서화되어 있으며 팀원에게 공유됨',
   '정의된 정책이 모든 보안 그룹에 